In [ ]:
import os
import numpy as np
import networkx as nx


# Load Niklas data.
data_folder = "/home/cedric/PHD/Dev/VolcapySIAM/data/InversionDatas/stromboli_173018/"

data_coords = np.load(os.path.join(data_folder,"niklas_data_coords_corrected_final.npy"))
data_coords_inds_insurf = np.load(os.path.join(data_folder,"niklas_data_inds_insurf.npy"))[1:] # Remove base station.
surface_data_coords = np.load(os.path.join(data_folder,"surface_data_coords.npy"))

# Get the nodes coordinates. Remember the nodes store their corresponding data
# index in the 'ind' attribute.
xyz = surface_data_coords
mean = np.mean(xyz, axis=0)
std = np.std(xyz, axis=0)
xyz_norm = (xyz - mean) / std

# Scale the z coordinate.
xyz_norm[:, 2] = 0.5 * xyz_norm[:, 2]

In [ ]:
# Load graphs.
graph_surface = nx.read_edgelist(os.path.join(data_folder, "stromboli_surface.edgelist"))
graph_trails = nx.read_edgelist(os.path.join(data_folder, "stromboli_trails.edgelist"))

# The nodes are currently labeled by strings. Set them to
# integers, so can be used to index datapoints.
graph_surface = nx.relabel_nodes(graph_surface, lambda x: int(x))
graph_trails = nx.relabel_nodes(graph_trails, lambda x: int(x))

# Note that the nodes in the trail graph are labelled by their position in the field campaing data 'data_coords'. 
# To match them with the nodes in the discretisation in the surface, we have to relabel them.
graph_trails_insurf = nx.relabel_nodes(graph_trails, lambda i: data_coords_inds_insurf[i])

graph_merged = nx.compose(graph_surface, graph_trails_insurf)

In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd
import numpy as np

plt.rcParams["figure.figsize"] = (60,60)


x, y, z = xyz_norm[:, 0], xyz_norm[:, 1], xyz_norm[:, 2]

trace1 = go.Scatter3d(
    x=x[data_coords_inds_insurf[graph_trails.nodes]], y=y[data_coords_inds_insurf[graph_trails.nodes]], z=z[data_coords_inds_insurf[graph_trails.nodes]],
    mode='markers',
    marker=dict(
        size=4,
        color=z[graph_trails.nodes],
        colorscale='RdBu_r',
    ))
traces_list = [trace1]

x_lines, y_lines, z_lines = [], [], []
for edge in list(graph_trails.edges):
    for i in range(2):
        x_lines.append(xyz_norm[data_coords_inds_insurf[edge[i]], 0])
        y_lines.append(xyz_norm[data_coords_inds_insurf[edge[i]], 1])
        z_lines.append(xyz_norm[data_coords_inds_insurf[edge[i]], 2])
    x_lines.append(None)
    y_lines.append(None)
    z_lines.append(None)
    
trace2 = go.Scatter3d(
    x=x_lines,
    y=y_lines,
    z=z_lines,
    mode='lines',
    name='lines',
    line=dict(
        color='yellow',
        width=8
    )
)

fig = go.Figure(data=[trace1, trace2])
fig.show()

In [ ]:
# Check if the relabelling went Ok.
trace1 = go.Scatter3d(
    x=x[graph_trails_insurf.nodes], y=y[graph_trails_insurf.nodes], z=z[graph_trails_insurf.nodes],
    mode='markers',
    marker=dict(
        size=4,
        color=z[graph_trails.nodes],
        colorscale='RdBu_r',
    ))
traces_list = [trace1]

x_lines, y_lines, z_lines = [], [], []
for edge in list(graph_trails_insurf.edges):
    for i in range(2):
        x_lines.append(xyz_norm[edge[i], 0])
        y_lines.append(xyz_norm[edge[i], 1])
        z_lines.append(xyz_norm[edge[i], 2])
    x_lines.append(None)
    y_lines.append(None)
    z_lines.append(None)
    
trace2 = go.Scatter3d(
    x=x_lines,
    y=y_lines,
    z=z_lines,
    mode='lines',
    name='lines',
    line=dict(
        color='yellow',
        width=8
    )
)

fig = go.Figure(data=[trace1, trace2])
fig.update_layout(
    showlegend=False,
    autosize=False,
    width=1800,
    height=1200,
    margin=dict(
        l=50,
        r=50,
        b=20,
        t=40,
        pad=2
    ),
)
fig.show()

In [ ]:
# Plot the whole surface, with the trails in color.

# Built color array for the edges.
edge_colors = []
for node1, node2, edge_attr in list(graph_trails_insurf.edges(data=True)):
    attr = edge_attr['path_attribute']
    if attr == 'no_trail':
        edge_colors.append('green')
        edge_colors.append('green')
        edge_colors.append('green')

    elif attr == 'trail':
        edge_colors.append('yellow')
        edge_colors.append('yellow')
        edge_colors.append('yellow')
    elif attr == 'override':
        edge_colors.append('red')
        edge_colors.append('red')
        edge_colors.append('red')
len(edge_colors)

trace1 = go.Scatter3d(
    x=x[graph_merged.nodes], y=y[graph_merged.nodes], z=z[graph_merged.nodes],
    mode='markers',
    marker=dict(
        size=1,
        color=z[graph_merged.nodes],
        colorscale='RdBu_r',
    ))
traces_list = [trace1]

x_lines, y_lines, z_lines = [], [], []
for edge in list(graph_trails_insurf.edges):
    for i in range(2):
        x_lines.append(xyz_norm[edge[i], 0])
        y_lines.append(xyz_norm[edge[i], 1])
        z_lines.append(xyz_norm[edge[i], 2])
    x_lines.append(None)
    y_lines.append(None)
    z_lines.append(None)
    
trace2 = go.Scatter3d(
    x=x_lines,
    y=y_lines,
    z=z_lines,
    mode='lines',
    name='lines',
    line=dict(
        color=edge_colors,
        width=10
    )
)

fig = go.Figure(data=[trace1, trace2])
fig.update_layout(
    showlegend=False,
    autosize=False,
    width=1800,
    height=1200,
    margin=dict(
        l=50,
        r=50,
        b=20,
        t=40,
        pad=2
    ),
)
fig.show()

## Try the shortest path algorithm.

In [ ]:
# Plot shortest path.
from volcapy.graph.cost_functions import walking_cost_fn

# Base cell.
base_cell = data_coords_inds_insurf[0]

shortest_path_inds = nx.shortest_path(graph_merged, base_cell, 3800, weight=walking_cost_fn)

def compute_path_cost(graph, path, cost_fn):
    cost = 0
    for i in range(len(path) - 1):
        cost += cost_fn(path[i], path[i + 1], graph.get_edge_data(path[i], path[i + 1]))
    return cost

print("Time for shortest path: {} mins.".format(compute_path_cost(graph_merged, shortest_path_inds, walking_cost_fn) / 60.0 ))
        

# First plot all nodes.
trace1 = go.Scatter3d(
    x=x[graph_merged.nodes], y=y[graph_merged.nodes], z=z[graph_merged.nodes],
    mode='markers',
    marker=dict(
        size=2,
        color=z[graph_merged.nodes],
        colorscale='RdBu_r',
    ))
traces_list = [trace1]

# Plot path.
x_lines, y_lines, z_lines = [], [], []
for edge in list(graph_trails_insurf.edges):
    for i in range(2):
        x_lines.append(xyz_norm[edge[i], 0])
        y_lines.append(xyz_norm[edge[i], 1])
        z_lines.append(xyz_norm[edge[i], 2])
    x_lines.append(None)
    y_lines.append(None)
    z_lines.append(None)
    
trace2 = go.Scatter3d(
    x=x[shortest_path_inds],
    y=y[shortest_path_inds],
    z=z[shortest_path_inds],
    mode='lines',
    name='lines',
    line=dict(
        color='yellow',
        width=15
    )
)

fig = go.Figure(data=[trace1, trace2])
fig.update_layout(
    showlegend=False,
    autosize=False,
    width=1800,
    height=1200,
    margin=dict(
        l=50,
        r=50,
        b=20,
        t=40,
        pad=2
    ),
)
fig.show()